# Solving the Quadratic Assignment Problem (QAP) using QAOA

There are three implementation methods we have tried. The first is identical to our VQE implementation but without a variational form. The second uses MinimumEigenOptimiser. The third uses RecursiveMinimumEigenOptimiser. Unfortunately , even with high maxiter values on multiple classical optimisers with varied p values, we have had no feasible solutions. For this notebook each example will use one set of parameters just to visualise the approach.

The goal of this notebook was to acheive good quality feasible solutions on the Aer backends such that an optimal point is used that can become the initial point when the same instance is run on the quantum devices. This was the approach attempted for VQE.

In [1]:
#Initialisation cell
from docplex.mp.model import Model

from qiskit import BasicAer, Aer
from qiskit.aqua.algorithms import VQE, ExactEigensolver,NumPyEigensolver,QAOA,NumPyMinimumEigensolver
from qiskit.aqua.components.optimizers import SPSA, COBYLA,ADAM, L_BFGS_B
from qiskit.aqua import QuantumInstance

from qiskit.optimization import QuadraticProgram
from qiskit.optimization.applications.ising import docplex
from qiskit.optimization.algorithms import MinimumEigenOptimizer,RecursiveMinimumEigenOptimizer

# setup aqua logging
import logging
from qiskit.aqua import set_qiskit_aqua_logging

import numpy as np
import pandas as pd

from qiskit.providers.aer import QasmSimulator

### QAP setup

In [2]:
def CSVtoNumpyArray(rawdata):
    """
    Input: 
    rawdata = a csv file (insert name as a string)

    Output:
    two numpy matrices in a tuple
    """
    data = pd.read_csv(rawdata)  #Reads the data in as a pandas object
    c = data.columns
    column = int(c[0])
    final_data1 = data.iloc[:column,:].values  #Sets data into a series of numpy arrays of strings
    final_data2 = data.iloc[column:,:].values  #1 is for the first matrix(loc) and 2 is for the second(flow)
    

    #Forms the matrix as a numpy array (easier to work with) instead of an list of lists of strings
    def string_to_integers(final_data):
        matrix = []
        for j in range(column):
            string = final_data[j][0]
            string2 = string.split(" ")
            emptyarray = []
            for i in string2:
                if i != '':
                    emptyarray.append(int(i))
            matrix.append(emptyarray)
        npmatrix = np.array(matrix) 
        return npmatrix
    return string_to_integers(final_data1),string_to_integers(final_data2)

In [3]:
def qap_value(z, MatrixLoc, MatrixFlow):
    """Compute the TSP value of a solution.
    Args:
        z (list[int]): list of allocations
        MatrixLoc (numpy array): matrix of distances
        MatrixFlow (numpy array): matrix of flow
    Returns:
        float: value of the QAP
    """
    matrix_length = len(MatrixLoc)
    x = np.reshape(z, (matrix_length,matrix_length))
    
    total = 0
    for i in range(matrix_length):
        for j in range(matrix_length):
            for k in range(matrix_length):
                for p in range(matrix_length):
                        total += MatrixLoc[i,j]* MatrixFlow[k,p]*x[i,k]*x[j,p]
    
    return total

### Feasibility functions

In [4]:
def qap_feasible(x):
    """Check whether a solution is feasible or not.

    Args:
        x (numpy.ndarray) : binary string as numpy array.

    Returns:
        bool: feasible or not.
    """
    n = int(np.sqrt(len(x)))
    y = np.reshape(x, (n,n))
   
    for i in range(n):
        if sum(y[i, p] for p in range(n)) != 1:
            return False
    for p__ in range(n):
        if sum(y[i, p__] for i in range(n)) != 1:
            return False
    return True

In [5]:
def choose_best_feasible(eigenstates):
    """
    Input:
    eigenstates = dictionary
    
    Output:
    feasible binary 1D numpyarray
    probability of this answer
    
    """
    bestinarray = sorted(eigenstates.items(), key=lambda item: item[1])[::-1]
    feasible = False
    counter = 0
    total = sum(eigenstates.values())
    
    feasible=False
    while feasible==False and counter<len(bestinarray):
        #string to array
        bestasint = np.array([0])
        for i in bestinarray[counter][0]:
            bestasint = np.hstack((bestasint, int(i)))
        feasible = qap_feasible(bestasint[1:])
        frequency = bestinarray[counter][1]
        counter += 1
        
    if feasible == False:
        return feasible
    else:
        return bestasint[1:], frequency/total

## 1. QAOA like VQE

In [6]:
def testing_quantum1(ins, maximum_iterations, p_value):
    """
    Input:
    ins: name of instance (str)
    maximum_iterations: maxiter for classical optimiser (int)
    p_value: for QAOA(int)
    
    Output:
    outmatrix : [soln,result time,eigenvector]
    """
    
    #get matrix
    datamatrix = CSVtoNumpyArray(ins)
    MatrixLoc = datamatrix[0]
    MatrixFlow = datamatrix[1]
    n = len(MatrixLoc)

    # Create an instance of a model and variables.
    thename = "qap" + str(n)
    mdl = Model(name=thename)
    x = {(i,p): mdl.binary_var(name='x_{0}_{1}'.format(i,p)) for i in range(n) for p in range(n)}

    # Object function
    qap_func = mdl.sum(MatrixLoc[i,j]* MatrixFlow[k,p]*x[i,k]*x[j,p] for i in range(n) for j in range(n) for p in range(n) for k in range(n))
    mdl.minimize(qap_func)

    # Constraints
    for i in range(n):
        mdl.add_constraint(mdl.sum(x[(i,p)] for p in range(n)) == 1)
    for p in range(n):
        mdl.add_constraint(mdl.sum(x[(i,p)] for i in range(n)) == 1)
    print(mdl.export_to_string())
    qubitOp_docplex, offset_docplex = docplex.get_operator(mdl) #potential error

    #QAOA
    machinesolutions = []
    seed = 10598
    spsa = SPSA(maxiter=maximum_iterations)
    sim = Aer.get_backend('qasm_simulator')
    #sim = QasmSimulator()
    backend_options = {"method": "matrix_product_state"}
    quantum_instance = QuantumInstance(backend=sim,seed_simulator=seed,
                                       seed_transpiler=seed, skip_qobj_validation = False, shots = 1,
                                       backend_options=backend_options)
    
    #THE UNIQUE STUFF-----------------------------------------------
    qaoa = QAOA(qubitOp_docplex,optimizer=spsa, p=p_value, quantum_instance=quantum_instance,include_custom=True)
    result = qaoa.run(backend=Aer.get_backend('qasm_simulator'))
    
    print('QAOA vector:', result['eigenstate'])
    print('QAOA time:', result['optimizer_time'])
    print('QAOA QAP objective:', result['eigenvalue'] + offset_docplex)
    x = choose_best_feasible(result['eigenstate'])
    print(x)
    
    if x == False:
        soln = 'No solution' #note fails if false
    else:
        soln = qap_value(x[0],MatrixFlow,MatrixLoc) 
    machinesolutions.append([soln,result['optimizer_time'],x])
    #--------------------------------------------------------------
    
    print(machinesolutions)
    return machinesolutions
    

In [7]:
ins = "made3.csv"
maximum_iterations= 1000
p_value= 5
ans1 = testing_quantum1(ins, maximum_iterations, p_value)

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: qap3

Minimize
 obj: [ 4704 x_0_0*x_1_1 + 3444 x_0_0*x_1_2 + 21280 x_0_0*x_2_1
      + 15580 x_0_0*x_2_2 + 4704 x_0_1*x_1_0 + 504 x_0_1*x_1_2
      + 21280 x_0_1*x_2_0 + 2280 x_0_1*x_2_2 + 3444 x_0_2*x_1_0
      + 504 x_0_2*x_1_1 + 15580 x_0_2*x_2_0 + 2280 x_0_2*x_2_1
      + 18368 x_1_0*x_2_1 + 13448 x_1_0*x_2_2 + 18368 x_1_1*x_2_0
      + 1968 x_1_1*x_2_2 + 13448 x_1_2*x_2_0 + 1968 x_1_2*x_2_1 ]/2
Subject To
 c1: x_0_0 + x_0_1 + x_0_2 = 1
 c2: x_1_0 + x_1_1 + x_1_2 = 1
 c3: x_2_0 + x_2_1 + x_2_2 = 1
 c4: x_0_0 + x_1_0 + x_2_0 = 1
 c5: x_0_1 + x_1_1 + x_2_1 = 1
 c6: x_0_2 + x_1_2 + x_2_2 = 1

Bounds
 0 <= x_0_0 <= 1
 0 <= x_0_1 <= 1
 0 <= x_0_2 <= 1
 0 <= x_1_0 <= 1
 0 <= x_1_1 <= 1
 0 <= x_1_2 <= 1
 0 <= x_2_0 <= 1
 0 <= x_2_1 <= 1
 0 <= x_2_2 <= 1

Binaries
 x_0_0 x_0_1 x_0_2 x_1_0 x_1_1 x_1_2 x_2_0 x_2_1 x_2_2
End

QAOA vector: {'111111011': 1}
QAOA time: 623.2947103977203
QAOA QAP objective: (504927.169

## 2. QAOA with MinimumEigenOptimiser

In [8]:
def testing_quantum2(ins, maximum_iterations, p_value):
    """
    Input:
    ins: name of instance (str)
    maximum_iterations: maxiter for classical optimiser (int)
    p_value: for QAOA(int)
    
    Output:
    outmatrix : [soln,eigenvector]
    """
    
    #get matrix
    datamatrix = CSVtoNumpyArray(ins)
    MatrixLoc = datamatrix[0]
    MatrixFlow = datamatrix[1]
    n = len(MatrixLoc)

    # Create an instance of a model and variables.
    thename = "qap" + str(n)
    mdl = Model(name=thename)
    x = {(i,p): mdl.binary_var(name='x_{0}_{1}'.format(i,p)) for i in range(n) for p in range(n)}

    # Object function
    qap_func = mdl.sum(MatrixLoc[i,j]* MatrixFlow[k,p]*x[i,k]*x[j,p] for i in range(n) for j in range(n) for p in range(n) for k in range(n))
    mdl.minimize(qap_func)

    # Constraints
    for i in range(n):
        mdl.add_constraint(mdl.sum(x[(i,p)] for p in range(n)) == 1)
    for p in range(n):
        mdl.add_constraint(mdl.sum(x[(i,p)] for i in range(n)) == 1)
    qubitOp_docplex, offset_docplex = docplex.get_operator(mdl) #potential error

    #QAOA
    machinesolutions = []
    seed = 10598
    spsa = SPSA(maxiter=maximum_iterations)
    sim = Aer.get_backend('qasm_simulator')
    #sim = QasmSimulator()
    backend_options = {"method": "matrix_product_state"}
    quantum_instance = QuantumInstance(backend=sim,seed_simulator=seed,
                                       seed_transpiler=seed, skip_qobj_validation = False, shots = 1,
                                       backend_options=backend_options)
    
    #THE UNIQUE STUFF------------------------------------------------------------
    mod = QuadraticProgram()
    mod.from_docplex(mdl)
    print(mod.export_as_lp_string())
    qaoa_mes = QAOA(quantum_instance=quantum_instance, include_custom=True, optimizer=spsa,p=p_value)
    qaoa = MinimumEigenOptimizer(qaoa_mes)
    qaoa_result = qaoa.solve(mod)
    print(qaoa_result)
    print('QAOA vector:', qaoa_result.x)
    x = qap_feasible(qaoa_result.x)
    print(x)
    
    if x == False:
        soln = 'No solution' #note fails if false
    else:
        soln = qap_value(x,MatrixFlow,MatrixLoc) #note fails if false
    machinesolutions.append([soln,x])
    #--------------------------------------------------------------
    
    print(machinesolutions)
    return machinesolutions

In [9]:
ins = "made3.csv"
maximum_iterations= 1000
p_value= 5
ans1 = testing_quantum2(ins, maximum_iterations, p_value)

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: qap3

Minimize
 obj: [ 4704 x_0_0*x_1_1 + 3444 x_0_0*x_1_2 + 21280 x_0_0*x_2_1
      + 15580 x_0_0*x_2_2 + 4704 x_0_1*x_1_0 + 504 x_0_1*x_1_2
      + 21280 x_0_1*x_2_0 + 2280 x_0_1*x_2_2 + 3444 x_0_2*x_1_0
      + 504 x_0_2*x_1_1 + 15580 x_0_2*x_2_0 + 2280 x_0_2*x_2_1
      + 18368 x_1_0*x_2_1 + 13448 x_1_0*x_2_2 + 18368 x_1_1*x_2_0
      + 1968 x_1_1*x_2_2 + 13448 x_1_2*x_2_0 + 1968 x_1_2*x_2_1 ]/2
Subject To
 c0: x_0_0 + x_0_1 + x_0_2 = 1
 c1: x_1_0 + x_1_1 + x_1_2 = 1
 c2: x_2_0 + x_2_1 + x_2_2 = 1
 c3: x_0_0 + x_1_0 + x_2_0 = 1
 c4: x_0_1 + x_1_1 + x_2_1 = 1
 c5: x_0_2 + x_1_2 + x_2_2 = 1

Bounds
 0 <= x_0_0 <= 1
 0 <= x_0_1 <= 1
 0 <= x_0_2 <= 1
 0 <= x_1_0 <= 1
 0 <= x_1_1 <= 1
 0 <= x_1_2 <= 1
 0 <= x_2_0 <= 1
 0 <= x_2_1 <= 1
 0 <= x_2_2 <= 1

Binaries
 x_0_0 x_0_1 x_0_2 x_1_0 x_1_1 x_1_2 x_2_0 x_2_1 x_2_2
End



constraint c0 is infeasible due to substitution
constraint c1 is infeasible due to substitution
constraint c2 is infeasible due to substitution
constraint c3 is infeasible due to substitution
constraint c4 is infeasible due to substitution
constraint c5 is infeasible due to substitution


optimal function value: 47238.0
optimal value: [1. 1. 1. 1. 1. 1. 0. 1. 1.]
status: SUCCESS
QAOA vector: [1. 1. 1. 1. 1. 1. 0. 1. 1.]
False
[['No solution', False]]


## 3. QAOA with RecursiveMinimumEigenOptimiser

In [10]:
def testing_quantum3(ins, maximum_iterations, p_value):
    """
    Input:
    ins: name of instance (str)
    maximum_iterations: maxiter for classical optimiser (int)
    p_value: for QAOA(int)
    
    Output:
    outmatrix : [soln,eigenvector]
    """
    
    #get matrix
    datamatrix = CSVtoNumpyArray(ins)
    MatrixLoc = datamatrix[0]
    MatrixFlow = datamatrix[1]
    n = len(MatrixLoc)

    # Create an instance of a model and variables.
    thename = "qap" + str(n)
    mdl = Model(name=thename)
    x = {(i,p): mdl.binary_var(name='x_{0}_{1}'.format(i,p)) for i in range(n) for p in range(n)}

    # Object function
    qap_func = mdl.sum(MatrixLoc[i,j]* MatrixFlow[k,p]*x[i,k]*x[j,p] 
                       for i in range(n) for j in range(n) for p in range(n) for k in range(n))
    mdl.minimize(qap_func)

    # Constraints
    for i in range(n):
        mdl.add_constraint(mdl.sum(x[(i,p)] for p in range(n)) == 1)
    for p in range(n):
        mdl.add_constraint(mdl.sum(x[(i,p)] for i in range(n)) == 1)
    qubitOp_docplex, offset_docplex = docplex.get_operator(mdl) #potential error

    #QAOA
    seed = 10598
    mod = QuadraticProgram()
    mod.from_docplex(mdl)
    print(mod.export_as_lp_string())
    spsa = SPSA(maxiter=maximum_iterations)
    sim = Aer.get_backend('qasm_simulator')
    #sim = QasmSimulator()
    backend_options = {"method": "matrix_product_state"}
    quantum_instance = QuantumInstance(backend=sim,seed_simulator=seed, seed_transpiler=seed, skip_qobj_validation = False, 
                                       shots = 1,backend_options=backend_options)
    qaoa_mes = QAOA(quantum_instance=quantum_instance, include_custom=True, optimizer=spsa,p=p_value)
    qaoa = MinimumEigenOptimizer(qaoa_mes)
    
    #THE UNIQUE PART(unique from 2)---------------------------------------------------------------------------------------
    rmachinesolutions = []
    exact_mes = NumPyMinimumEigensolver()
    exact = MinimumEigenOptimizer(exact_mes) 
    rqaoa = RecursiveMinimumEigenOptimizer(min_eigen_optimizer=qaoa, min_num_vars=1, min_num_vars_optimizer=exact)
    rqaoa_result = rqaoa.solve(mod)
    rx = qap_feasible(rqaoa_result.x)
    print(rx)
    if rx == False:
        rsoln = 'No solution' #note fails if false
    else:
        rsoln = qap_value(rx,MatrixFlow,MatrixLoc) #note fails if false
    rmachinesolutions.append([rsoln,rx,i])
    print(rmachinesolutions)
    return rmachinesolutions

In [11]:
ins = "made3.csv"
maximum_iterations= 1000
p_value= 5
ans1 = testing_quantum3(ins, maximum_iterations, p_value)

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: qap3

Minimize
 obj: [ 4704 x_0_0*x_1_1 + 3444 x_0_0*x_1_2 + 21280 x_0_0*x_2_1
      + 15580 x_0_0*x_2_2 + 4704 x_0_1*x_1_0 + 504 x_0_1*x_1_2
      + 21280 x_0_1*x_2_0 + 2280 x_0_1*x_2_2 + 3444 x_0_2*x_1_0
      + 504 x_0_2*x_1_1 + 15580 x_0_2*x_2_0 + 2280 x_0_2*x_2_1
      + 18368 x_1_0*x_2_1 + 13448 x_1_0*x_2_2 + 18368 x_1_1*x_2_0
      + 1968 x_1_1*x_2_2 + 13448 x_1_2*x_2_0 + 1968 x_1_2*x_2_1 ]/2
Subject To
 c0: x_0_0 + x_0_1 + x_0_2 = 1
 c1: x_1_0 + x_1_1 + x_1_2 = 1
 c2: x_2_0 + x_2_1 + x_2_2 = 1
 c3: x_0_0 + x_1_0 + x_2_0 = 1
 c4: x_0_1 + x_1_1 + x_2_1 = 1
 c5: x_0_2 + x_1_2 + x_2_2 = 1

Bounds
 0 <= x_0_0 <= 1
 0 <= x_0_1 <= 1
 0 <= x_0_2 <= 1
 0 <= x_1_0 <= 1
 0 <= x_1_1 <= 1
 0 <= x_1_2 <= 1
 0 <= x_2_0 <= 1
 0 <= x_2_1 <= 1
 0 <= x_2_2 <= 1

Binaries
 x_0_0 x_0_1 x_0_2 x_1_0 x_1_1 x_1_2 x_2_0 x_2_1 x_2_2
End



constraint c0 is infeasible due to substitution
constraint c1 is infeasible due to substitution
constraint c4 is infeasible due to substitution
constraint c5 is infeasible due to substitution


False
[['No solution', False, 2]]


## PS: Transpiler change

Another adjustment we made was to use:                                   

In [14]:
seed = 10598
sim = QasmSimulator("matrix_product_state")
quantum_instance = QuantumInstance(backend=sim,seed_simulator=seed, seed_transpiler=seed, skip_qobj_validation = False, shots = 1)

This was faster but didn't change feasibility

Please note we used an older version of qiskit in this notebook. An update did not change the quality of our results.

In [12]:
import qiskit.tools.jupyter
%qiskit_version_table